In [1]:
#We can go into our root file and see what Trees are availiable
%matplotlib inline
import sys, os
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
    sys.path.append(os.path.realpath("../../"))
import numpy as np
import pandas as pd
import ntpath
import glob
import deepconfig

from keras.utils.visualize_util import plot
from IPython.display import Image, display

from CMS_SURF_2016.utils.preprocessing import ObjectProfile, \
        preprocessFromPandas_label_dir_pairs, label_dir_pairs_args_decoder, \
        genFrom_label_dir_pairs,maxMutualLength, start_num_fromSplits
from CMS_SURF_2016.utils.metrics import plot_history, print_accuracy_m
from CMS_SURF_2016.utils.callbacks import OverfitStopping, SmartCheckpoint
from CMS_SURF_2016.utils.archiving import *
from CMS_SURF_2016.layers.lorentz import Lorentz, _lorentz
from CMS_SURF_2016.layers.slice import Slice

from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Flatten, Reshape, Activation, Dropout, Convolution2D, merge, Input, Flatten, Lambda, LSTM, Masking
from keras.engine.topology import Layer
from keras.callbacks import EarlyStopping
from keras.utils.visualize_util import plot
from keras.layers.advanced_activations import LeakyReLU


dc = deepconfig.deepconfig(gpu='gpu0', backend='theano')




Using Theano backend.
Using gpu device 1: GeForce GTX TITAN X (CNMeM is disabled, cuDNN not available)

Couldn't import dot_parser, loading of dot files will not be possible.
using gpu1
using theano


In [2]:
#The observables taken from the table
observ_types = ['E/c', 'Px', 'Py', 'Pz', 'Charge', "PT_ET", "Eta", "Phi", "Dxy_Ehad_Eem"]
vecsize = len(observ_types)
epochs = 100
batch_size = 100
sample_start = 0
num_samples = 10000


object_profiles = [ObjectProfile("Electron",5),
                    ObjectProfile("MuonTight", 5),
                    ObjectProfile("Photon", 25),
                    ObjectProfile("MissingET", 1),
                    ObjectProfile("EFlowPhoton",100, sort_columns=["PT_ET"], sort_ascending=False),  #1300
                    ObjectProfile("EFlowNeutralHadron",100, sort_columns=["PT_ET"], sort_ascending=False),  #1000
                    ObjectProfile("EFlowTrack",100, sort_columns=["PT_ET"], sort_ascending=False)]  #1050


label_dir_pairs = \
            [   ("ttbar", "/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_unjoined/"),
                ("wjet", "/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_unjoined/"),
                ("qcd", "/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_unjoined/")
            ]
#ttbar_files = glob.glob("/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_unjoined/*.h5")
#WJet_files = glob.glob("/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_unjoined/*.h5")
#qcd_files = glob.glob("/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_unjoined/*.h5")

#files = {0:ttbar_files, 1:WJet_files}


In [3]:
def genModel(name,out_dim, depth, lstm_activation="relu", lstm_dropout = 0.0, dropout=0.0):
    inputs = []
    mergelist = []
    for i, profile in enumerate(object_profiles):
        # print(o)
        inp = a = Input(shape=(profile.max_size, vecsize), name="input_"+str(i))
        inputs.append(inp)
        #a = Flatten(name="flatten_"+str(i))(a)
        # b2 = Dense(10, activation='relu')(b2)
        mergelist.append(a)
    # print(mergelist)
    a = merge(mergelist,mode='concat',concat_axis=1, name="merge")
    # a = Flatten()(a)
    for i in range(depth):
        a = Masking(mask_value=0.0)(a)
        a = LSTM(vecsize,
                 input_shape=(None,vecsize),
                 dropout_W=lstm_dropout,
                 dropout_U=lstm_dropout,
                 activation=lstm_activation,
                 name = "lstm_" +str(i))(a)
        if(dropout > 0.0):
            a =  Dropout(dropout, name="dropout_"+str(i))(a)
    dense_out = Dense(out_dim, activation='sigmoid', name='main_output')(a)
    model = Model(input=inputs, output=dense_out, name=name)
    return model

In [1]:
archive_dir = "/data/shared/Delphes/keras_archive/"
earlyStopping = EarlyStopping(verbose=1, patience=10)
overfitStopping = OverfitStopping(verbose=1, patience=20)
trials = []
ldps_pairs = []
for i, j in [(2,0)]:
    ldps = [label_dir_pairs[i]] + [label_dir_pairs[j]]
    SNs = start_num_fromSplits((.7,.15,.15), maxMutualLength(label_dir_pairs, object_profiles))
    print(SNs)
    gen_lambda =  lambda s : (DataProcedure(archive_dir,
                                           False,
                                           genFrom_label_dir_pairs,
                                           start=s[0],
                                           samples_per_label=s[1],
                                           stride=10000,
                                            batch_size=100,
                                           archive_dir=archive_dir,
                                           label_dir_pairs=ldps,
                                           object_profiles=object_profiles,
                                           observ_types=observ_types),
                             len(ldps)*s[1])
    l = [gen_lambda(s) for s in SNs]
    train, num_train = l[0]
    val,   num_val   = l[1]
    test,  num_test  = l[2]
    
    
   
    #gg = train.getData()
    
#    count = 10
#    for X,Y in gg:
#        for Xi in X:
#            print(Xi.shape)
#        print('')
#        for Yi in Y:
#            print(Yi.shape)
#        print('')
#        count -= 1
#        if(count == 0): raise ValueError()
        
    
    labels = [x[0] for x in ldps]
    for name in ['LSTM']:
        for depth in [1]:
            #for width in [100]:
                for activation in ['tanh']:
                    for lstm_dropout in [0.0]:
                        for dropout in [0.0]:
                            activation_name = activation if isinstance(activation, str) \
                                                else activation.__name__

                            model = genModel(name, len(ldps), depth, activation, lstm_dropout, dropout)

                            print(model.summary())

                            trial = KerasTrial(archive_dir, name=name, model=model)

                            trial.setTrain(train_procedure=train,
                                           samples_per_epoch=num_train
                                          )
                            trial.setValidation(val_procedure=val,
                                               nb_val_samples=num_val)
                            trial.setCompilation(loss='binary_crossentropy',
                                      optimizer='rmsprop',
                                      metrics=['accuracy']
                                          )
                            trial.setFit_Generator( 
                                            nb_epoch=epochs,
                                            callbacks=[earlyStopping, overfitStopping])



                            print("EXECUTE: ", name,labels, depth, activation_name)
                            trial.execute(custom_objects={"Lorentz":Lorentz,"Slice": Slice},
                                         train_arg_decode_func=label_dir_pairs_args_decoder,
                                         val_arg_decode_func=label_dir_pairs_args_decoder)


                            trial.test(test_proc=test,
                                         test_samples=num_test,
                                         custom_objects={"Lorentz":Lorentz,"Slice": Slice},
                                        arg_decode_func = label_dir_pairs_args_decoder)



                            trial.to_record({"lables": labels,
                                             "depth": depth,
                                             #"width": width,
                                             "activation": activation_name,
                                             "dropout":dropout,
                                             "lstm_dropout":lstm_dropout
                                            })
                                       

                
        

    

IndentationError: unexpected indent (<ipython-input-1-c848c0f87c3a>, line 44)